# Fitting with scipy - solution to bonus exercise

In [ ]:
# import python modules for plotting, fitting
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
# for interactive widgets
import ipywidgets as ipw

In [ ]:
%matplotlib notebook

In [ ]:
def lorentzian(xx, scale=1.0, center=1.0, hwhm=3.0):
    if hwhm == 0:
        raise ValueError('hwhm of the lorentzian is equal to zero.')
    return scale * hwhm / ((xx-center)**2 + hwhm**2) / np.pi

In [ ]:
xx = np.linspace(-10, 10, 500)
new_data = lorentzian(xx, 3, 4, 0.5) * (
    1. + 0.1*np.random.normal(0,1,500)) + 0.01*np.random.normal(0,1,500)

initial_params = {"scale": 5.5,
                  "center": 1.0,
                  "hwhm": 2.5}

fig8 = plt.figure()
gs = fig8.add_gridspec(3, 1)
f8_ax1 = fig8.add_subplot(gs[0:2, :])
f8_ax2 = fig8.add_subplot(gs[2, :])
f8_ax1.plot(xx, new_data, label="reference data for exercise")
lines = f8_ax1.plot(xx, lorentzian(xx, *list(initial_params.values())), label='model to be fitted')
fit_lines = f8_ax1.plot(xx, np.zeros_like(xx), '--', label='fit')
res_lines = f8_ax2.plot(xx, np.zeros_like(xx), label='residuals')
f8_ax1.set_ylabel('lorentzian(x,{},{},{})'.format(*list(initial_params.values())))
f8_ax1.set_xlabel('x')
f8_ax1.grid()
f8_ax1.legend()
f8_ax2.set_xlabel('x')
f8_ax2.grid()
f8_ax2.legend()

# define slider to interactively modify the parameters
sliders = {key: ipw.FloatSlider(min=1.0, max=10.0, value=value, description=key)
           for key, value in initial_params.items()}
sliders["center"].min = -sliders["center"].max

# define function to be plotted
def interactive_plot(change):
    params = [sliders[key].value for key in sliders]
    lines[0].set_ydata(lorentzian(xx, *params))
    f8_ax1.set_ylabel('lorentzian(x,{},{},{})'.format(*params))

# add observers to the sliders to update the plot
for sl in sliders.values():
    sl.observe(interactive_plot, names="value")

# Define function to reset all parameters' values to the initial ones
def reset_values(b):
    """Reset the interactive plots to inital values."""
    for key, value in initial_params.items():
        sliders[key].value = value

# Define reset button and occurring action when clicking on it
reset_button = ipw.Button(description = "Reset")
reset_button.on_click(reset_values)

params = [0, 0, 0]
pcov = [0, 0, 0]

# Capture fit results output
fit_results = ipw.Output()

chosen_method_optim = ipw.RadioButtons(
    options=['lm', 'trf', 'dogbox'],
    value='lm', # Defaults to 'lm'
    description='Method for optimization',
    style={'description_width': 'initial'},
    disabled=False
)

# Define reset button and occurring action when clicking on it
run_fit_button = ipw.Button(description = "Fit!")

# display the interactive plot
display(ipw.VBox(list(sliders.values())), reset_button)
display(ipw.HBox([chosen_method_optim, run_fit_button, fit_results]))

def run_fit(button):
    params, pcov = curve_fit(lorentzian, xx, new_data,
                                      method=chosen_method_optim.value,
                                      p0=list(initial_params.values()))
    fit_results.clear_output()
    with fit_results:
        params_error = np.sqrt(np.diag(pcov))
        print('Values of refined parameters:')
        print('scale:', params[0],'+/-', params_error[0])
        print('center :', params[1],'+/-', params_error[1])
        print('HWHM', params[2],'+/-', params_error[2])
    fit_lines[0].set_ydata(lorentzian(xx, *params))
    res_lines[0].set_ydata(new_data - fit_lines[0].get_ydata())

run_fit_button.on_click(run_fit)